In [10]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.initializers import glorot_uniform

In [25]:
# This function reads in a .dcm file and checks the important fields for our device, and returns True if the data
# is valid and false otherwise.
def validate_dicom_file(filename): 
    print('Validate file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)  
    # check the validity of each DICOM file by analyzing the fields: image position, image type and body part.
    body_part_examined = ds.BodyPartExamined
    print(ds)
    print('body_part_examined:', body_part_examined)
    
    return body_part_examined == 'CHEST'
    
    
# This function reads in a .dcm file, and returns a numpy array
# of just the imaging data    
def get_pixel_array(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    return img
    
    
# This function takes the numpy array output by get_pixel_array and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    # todo
    proc_img = img.copy()
    proc_img = (proc_img - img_mean)/img_std
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        json_savedModel = json_file.read()
    model = tf.keras.models.model_from_json(json_savedModel)
    model.load_weights(weight_path)
    loss = 'binary_crossentropy'
    model.compile(loss=loss)    
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo 
    prediction = model.predict_classes(img) > threshold
    
    return prediction 

In [26]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.3535353535353536 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    is_valid = validate_dicom_file(i)
    
    if not is_valid:
        continue
    
    img = get_pixel_array(i)
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Validate file test1.dcm ...
(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patient ID                          LO: '2'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '81'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'PA'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.112507010803284478207522016832191866964708
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.112630850362182468372440828755218293352329
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROM

ValueError: Error when checking input: expected model_1_input to have 4 dimensions, but got array with shape (1024, 1024)